In [9]:
from aocd import get_data
puzzle_input = get_data(day=13, year=2018).split('\n')

In [2]:
puzzle_input = r"""/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """.split('\n')

In [39]:
from collections import defaultdict
import operator
import re


class CollisonError(Exception):
    pass


class Cart:
    
    direction_map = {
        'v': (0, 1),
        '^': (0, -1),
        '<': (-1, 0),
        '>': (1, 0)
    }  # Work out how to move based on the current direction
    
    turn_map = {  # For \ and / junctions
        '\\': {
            'v': '>',
            '<': '^',
            '>': 'v',
            '^': '<'
        },
        '/': {
            'v': '<',
            '<': 'v',
            '>': '^',
            '^': '>'
        }
    }
    
    ordered_turns = '^<v>'  # For + junctions
    
    def __init__(self, char, x, y, cart_id, x_lim):
        self.x = x
        self.y = y
        self.x_lim = x_lim  # Needed to get a sort ID
        self.direction_char = char
        self.turn_count = 0
        self.id = cart_id
        self.is_alive = True
        
    def __repr__(self):
        return 'Cart {} ({}: {}, {})'.format(self.id, self.direction_char, self.x, self.y)
        
    def get_coord_tuple(self):
        return self.x, self.y
        
    def step(self, grid_obj):
        # Move
        direction = self.direction_map[self.direction_char]
        self.x, self.y = tuple(map(operator.add, self.get_coord_tuple(), direction))

        # Some sanity checks
        if self.x > grid_obj.x_lim or self.x < 0:
            raise ValueError('x-value of cart {} outside area: {} at step {}'.format(self.id, self.x, grid_obj.steps))
            
        if self.y > grid_obj.y_lim or self.y < 0:
            raise ValueError('y-value of cart {} outside area: {} at step {}'.format(self.id, self.y, grid_obj.steps))
    
    def turn(self, grid_obj):
        # Orient
        grid_char = grid_obj.grid[self.x][self.y]
        
        if grid_char in ('/', '\\'):
            self.direction_char = self.turn_map[grid_char][self.direction_char]
        elif grid_char == '+':
            if self.turn_count == 0:
                self.direction_char = self.ordered_turns[(self.ordered_turns.find(self.direction_char)+1)%4]
            elif self.turn_count == 2:
                self.direction_char = self.ordered_turns[(self.ordered_turns.find(self.direction_char)-1)%4]
            
            self.turn_count = (self.turn_count + 1) % 3     
            
    def get_sort_id(self):
        return self.x_lim * self.y + self.x


class Grid:
    def __init__(self, puzzle_input):
        self.grid = defaultdict(lambda: defaultdict(str))
        self.carts = []
        self.x_lim = len(puzzle_input[0])
        self.y_lim = len(puzzle_input)
        self.parse_input(puzzle_input)
        self.steps = 0
        
        # Check for initial collisions
        for cart in self.carts:
            self.check_for_collisions(cart)
            
    def parse_input(self, puzzle_input):
        carts = '<>^v'
        regex = re.compile('[{}\/\\\\+]'.format(carts))
        
        for y_idx, line in enumerate(puzzle_input):
            for item in regex.finditer(line):
                x_idx = item.start()
                char = item.group()
                if char in carts:
                    self.carts.append(Cart(char, x_idx, y_idx, len(self.carts), self.x_lim))
                else:
                    self.grid[x_idx][y_idx] = char
        
    def check_for_collisions(self, cart):
        cart_map = defaultdict(list)
        
        for other_cart in self.carts:
            if cart == other_cart or not other_cart.is_alive:
                continue
            cart_map[other_cart.get_coord_tuple()].append(other_cart)
        
        cart_pos = cart.get_coord_tuple()
        cart_map[cart_pos].append(cart)
    
        collision_carts = cart_map[cart_pos] if len(cart_map[cart_pos]) > 1 else []
        if collision_carts:
            print('Turn {}\t: {}'.format(self.steps, '\thit\t'.join(str(x) for x in collision_carts[::-1])))
        
        for dead in collision_carts:
            dead.is_alive = False
            
    def alive_carts(self):
        return [cart for cart in self.carts if cart.is_alive]
                
    def step(self):
        for cart in self.carts:
            cart.moved = False
            
        for cart in sorted(self.carts, key=lambda cart: cart.get_coord_tuple()):
            if not cart.is_alive:
                continue
            
            cart.step(self)
            grid.check_for_collisions(cart)
            cart.turn(self)
        self.steps += 1        


grid = Grid(puzzle_input)
while len(grid.alive_carts()) > 1:
    grid.step()

print('\nThe winner is:')
print(grid.alive_carts()[0])

Turn 134	: Cart 3 (v: 32, 8)	hit	Cart 11 (>: 32, 8)
Turn 245	: Cart 9 (v: 99, 112)	hit	Cart 10 (^: 99, 112)
Turn 263	: Cart 8 (^: 123, 41)	hit	Cart 6 (v: 123, 41)
Turn 498	: Cart 7 (<: 47, 85)	hit	Cart 2 (>: 47, 85)
Turn 1016	: Cart 0 (<: 137, 54)	hit	Cart 14 (>: 137, 54)
Turn 1259	: Cart 12 (^: 11, 10)	hit	Cart 4 (v: 11, 10)
Turn 1501	: Cart 5 (<: 34, 39)	hit	Cart 13 (>: 34, 39)
Turn 10632	: Cart 15 (>: 26, 38)	hit	Cart 1 (^: 26, 38)

The winner is:
Cart 16 (v: 38, 38)
